# Mock mid-term exam: April 2023

The objectives of this mock mid-term exam are:
- to gain acquaintance with the type of programming questions that you will be asked during the mid-term exam
- Becoming more comfortable in performing Input-Output analysis calculations

We are accounting that you will need around 100-120 minutes to complete the assignment

Please ensure that you run each cell (ctrl+enter) so that your inputs are saved 

All questions have either coding cell or a text cell, or a combination of the two. 

You are allowed to create additional cells for each answer to facilitate your work  

## Data preparation: Download and import exiobase data - year 2015, industry-by-industry format

In [1]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import here your data
path = "data/IOT_2015_ixi/"
A = pd.read_csv(f'{path}A.txt', sep='\t', index_col=[0, 1], header=[0, 1])
Y = pd.read_csv(f'{path}Y.txt' , sep='\t', index_col=[0, 1], header=[0, 1])

# Import satellite accounts
F_sat = pd.read_csv(f'{path}satellite/F.txt' , sep='\t', index_col=[0], header=[0, 1])
F_sat_hh = pd.read_csv(f'{path}satellite/F_y.txt' , sep='\t', index_col=[0], header=[0, 1])

In [3]:
print(*A.index, sep="\n")

('AT', 'Cultivation of paddy rice')
('AT', 'Cultivation of wheat')
('AT', 'Cultivation of cereal grains nec')
('AT', 'Cultivation of vegetables, fruit, nuts')
('AT', 'Cultivation of oil seeds')
('AT', 'Cultivation of sugar cane, sugar beet')
('AT', 'Cultivation of plant-based fibers')
('AT', 'Cultivation of crops nec')
('AT', 'Cattle farming')
('AT', 'Pigs farming')
('AT', 'Poultry farming')
('AT', 'Meat animals nec')
('AT', 'Animal products nec')
('AT', 'Raw milk')
('AT', 'Wool, silk-worm cocoons')
('AT', 'Manure treatment (conventional), storage and land application')
('AT', 'Manure treatment (biogas), storage and land application')
('AT', 'Forestry, logging and related service activities (02)')
('AT', 'Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)')
('AT', 'Mining of coal and lignite; extraction of peat (10)')
('AT', 'Extraction of crude petroleum and services related to crude oil extraction, excluding surveying')
('AT', 'Extract

## Question 1: Carbon footprint of the nations (18 points)

#### 1.1 What were the per capita carbon footprint of each of the 49 exiobase regions when considering only CO2 emissions in 2015? (8 points)

In [4]:
# Import population data
population = pd.read_excel('data/exiobase_PopulationGDP_1995_2019.xlsx',sheet_name='Population', index_col=[0, 1, 2])
pop2015 = population.loc[:,"y2015"]

In [5]:
# Calculate the rest of MRIO variables
I = np.identity(A.shape[0])
L = np.linalg.inv(I-A)
x = L @ Y.sum(axis=1)

In [6]:
x_ = x.copy()
x_[x_!=0] = 1/x_[x_!=0]
inv_diag_x_ = np.diag(x_)

In [7]:
# CO2 Extensions 
F_sat_CO2 = F_sat[F_sat.index.str.contains("CO2")].sum(axis=0)
F_sat_hh_CO2 = F_sat_hh[F_sat_hh.index.str.contains("CO2")].sum(axis=0)

by running the following

F_sat_hh.index.str.contains("CO2")

You are obtaining a boolean array (i.e., an array of true and false values), where true is assigned to the position of all labels that contain "CO2" in them.

This is the equivalent of creating a list of all the labels concerning CO2 emissions in the extension such as

```
['CO2 - combustion - air',
'CO2 - non combustion - Cement production - air',
'CO2 - non combustion - Lime production - air',
'CO2 - agriculture - peat decay - air',
'CO2 - waste - biogenic - air', 'CO2 - waste - fossil - air']
```

##### Explanation

**F_sat_gg.index** returns all labels of your index

**.str** allows to treat each entry of the index as a string

**.contains()** is a method for strings that allows you check whether a string may contain another string  

In [8]:
# CO2 Intensities
f_sat_CO2 = F_sat_CO2 @ inv_diag_x_ 

In [9]:
# Aggregate Y by region
Y_reg = Y.groupby(level=0, axis=1, sort=False).sum()

In [10]:
# Aggregate F_sat_hh by region
F_sat_hh_CO2_reg = F_sat_hh_CO2.groupby(level=0, sort=False).sum()

In [11]:
# Footprint calculation
e_CO2 = f_sat_CO2 @ L @ Y_reg + F_sat_hh_CO2_reg
e_CO2_pp = e_CO2/pop2015.values/1000 #convert the unit from kg to metric ton/capita

e_CO2_pp

region
AT     9.811567
BE    11.808848
BG     5.072057
CY     9.815370
CZ     9.619117
DE    11.598026
DK    11.675322
EE    11.375827
ES     6.256477
FI    10.289243
FR     6.922484
GR    12.511011
HR     3.992216
HU     5.451145
IE    11.723657
IT     7.667742
LT     6.752614
LU    23.166439
LV     6.043555
MT    12.926639
NL    12.321761
PL     7.399379
PT     5.492885
RO     3.829157
SE     7.183384
SI     7.526126
SK     7.693971
GB     9.769436
US    18.518329
JP    10.047756
CN     6.435537
CA    14.149816
KR    12.219827
BR     2.466260
IN     1.629168
MX     3.494032
RU     7.964158
AU    18.637680
CH    11.225801
TR     5.136483
TW     6.871530
NO    13.540860
ID     1.843218
ZA     5.566805
WA     2.064527
WL     3.585200
WE     3.846268
WF     0.859331
WM     6.512545
dtype: float64

In [12]:
# Other method to aggregate results instead of groupby
# 1st collect the labels (r=number of regions; s=number of sectors)
Y_labels = Y.index.to_frame(index=None)
region_labels = Y_labels.region.drop_duplicates().reset_index(drop=True)
sector_labels = Y_labels.sector.drop_duplicates().reset_index(drop=True)
r = region_labels.shape[0]
s = sector_labels.shape[0]

In [13]:
# Create empty arrays of the desired shape/order
Yt = np.zeros((r*s,r))
F1_hht = np.zeros((F_sat_hh.shape[0],r))

In [14]:
# Aggregate Y and F_hh

for l in region_labels.items(): 
    Yt[:,l[0]]=Y.loc[:,l[1]].sum(axis=1)
    F1_hht[:,l[0]]=F_sat_hh.loc[:,l[1]].sum(axis=1)

In [15]:
np.allclose(Yt.sum(axis=0), Y_reg.sum(axis=0))

True

#### 1.2 What were the per capita carbon footprint of each of the 49 regions in CO2e when considering CH4, N2O, and CO2? (6 points)
Instruction: Please use 29.8 and 273 for GWP100 of CH4 and N2O, respectively.

In [16]:
F_sat_CO2 = F_sat[F_sat.index.str.contains("CO2")].sum(axis=0)
F_sat_CH4 = F_sat[F_sat.index.str.contains("CH4")].sum(axis=0)*29.8
F_sat_N2O = F_sat[F_sat.index.str.contains("N2O")].sum(axis=0)*273

F_GHG_ = pd.concat([F_sat_CO2, F_sat_CH4, F_sat_N2O],axis=1).T
F_GHG_.index = ["CO2", "CH4", "N2O"]
F_GHG_

region                        AT                       \
sector Cultivation of paddy rice Cultivation of wheat   
CO2                          0.0         2.290468e+08   
CH4                          0.0         3.193777e+05   
N2O                          0.0         2.788491e+08   

region                                   \
sector Cultivation of cereal grains nec   
CO2                        2.359974e+08   
CH4                        4.474301e+05   
N2O                        4.180574e+08   

region                                                                  \
sector Cultivation of vegetables, fruit, nuts Cultivation of oil seeds   
CO2                              9.388639e+07             4.644045e+07   
CH4                              2.838360e+05             1.270425e+05   
N2O                              9.107078e+07             1.288437e+08   

region                                        \
sector Cultivation of sugar cane, sugar beet   
CO2                             1.408942e+07   
CH4                             4.745501e+04   
N2O                             4.933941e+07   

region                                                             \
sector Cultivation of plant-based fibers Cultivation of crops nec   
CO2                         39418.948028             1.808811e+06   
CH4                           136.651618             1.656764e+04   
N2O                        450938.409455             4.648129e+06   

region                               ...                       WM  \
sector Cattle farming  Pigs farming  ... Landfill of waste: Paper   
CO2      1.886045e+08  6.049997e+07  ...             5.266399e+07   
CH4      2.651895e+09  1.441410e+08  ...             4.759316e+10   
N2O      5.799846e+08  1.647127e+08  ...             4.897483e+04   

region                                                                      \
sector Landfill of waste: Plastic Landfill of waste: Inert/metal/hazardous   
CO2                  4.062944e+07                             5.042184e+07   
CH4                  1.436608e+09                             3.059326e+07   
N2O                  2.444897e+04                             5.636156e+04   

region                                                      \
sector Landfill of waste: Textiles Landfill of waste: Wood   
CO2                   3.842686e+07            5.549526e+07   
CH4                   6.331344e+09            1.399794e+09   
N2O                   1.457448e+04            1.970197e+04   

region                                                    \
sector Activities of membership organisation n.e.c. (91)   
CO2                                         2.129266e+08   
CH4                                         2.671610e+06   
N2O                                         3.448882e+05   

region                                                      \
sector Recreational, cultural and sporting activities (92)   
CO2                                          4.679020e+08    
CH4                                          5.699469e+06    
N2O                                          8.085240e+05    

region                                \
sector Other service activities (93)   
CO2                     3.349103e+08   
CH4                     3.227710e+06   
N2O                     7.122501e+05   

region                                                \
sector Private households with employed persons (95)   
CO2                                     6.326304e+07   
CH4                                     9.343112e+05   
N2O                                     1.045804e+05   

region                                             
sector Extra-territorial organizations and bodies  
CO2                                           0.0  
CH4                                           0.0  
N2O                                           0.0  

[3 rows x 7987 columns]

In [17]:
# Intensities satellite
f_GHG = F_GHG_ @ inv_diag_x_ 
f_GHG

,0,1,2,3,4,5,6,7,8,9,...,7977,7978,7979,7980,7981,7982,7983,7984,7985,7986
CO2,0.0,691259.800152,310277.849577,66286.364256,316544.074316,167373.122947,26977.557519,5761.174473,3.834245e+05,73824.712479,...,3.158695e+04,4.388930e+04,28723.374473,8.355061e+04,9.429815e+04,14226.117991,10278.420127,13103.352764,10560.562008,0.0
CH4,0.0,963.876994,588.259219,200.395987,865.938182,563.734657,93.521696,52.768967,5.391183e+06,175887.120618,...,2.854555e+07,1.551873e+06,17427.794623,1.376609e+07,2.378545e+06,178.496429,125.200438,126.284045,155.965490,0.0
N2O,0.0,841562.290311,549641.469482,64298.463925,878215.055451,586120.113971,308613.433066,14804.579548,1.179082e+06,200989.676186,...,2.937425e+01,2.641061e+01,32.107002,3.168894e+01,3.347781e+01,23.042775,17.760877,27.866759,17.457712,0.0


In [18]:
# Households satellites
F_sat_hh_CO2 = F_sat_hh[F_sat_hh.index.str.contains("CO2")].sum(axis=0)
F_sat_hh_CH4 = F_sat_hh[F_sat_hh.index.str.contains("CH4")].sum(axis=0)*29.8
F_sat_hh_N2O = F_sat_hh[F_sat_hh.index.str.contains("N2O")].sum(axis=0)*273

F_hh_GHG_ = pd.concat([F_sat_hh_CO2, F_sat_hh_CH4, F_sat_hh_N2O],axis=1).T
F_hh_GHG_.index = ["CO2", "CH4", "N2O"]

# Aggregate F_sat_hh by region
F_hh_GHG_reg = F_hh_GHG_.groupby(level=0, axis=1, sort=False).sum()

F_hh_GHG_reg

region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,TR,TW,NO,ID,ZA,WA,WL,WE,WF,WM
CO2,1.649435e+10,2.346101e+10,4.402743e+09,1.488962e+09,1.120496e+10,2.021225e+11,8.171912e+09,7.055757e+08,6.308866e+10,5.450621e+09,...,4.498783e+10,2.006928e+10,3.764177e+09,6.478355e+10,4.836168e+10,2.564473e+11,1.732998e+11,3.611237e+10,2.639215e+11,3.301295e+11
CH4,4.648808e+08,9.236450e+07,2.681356e+08,1.941141e+06,7.526603e+08,1.179378e+09,2.794978e+08,7.700563e+07,4.143630e+08,4.219064e+08,...,6.013385e+08,3.100029e+07,4.091918e+07,1.766731e+10,4.893146e+09,3.051683e+10,6.172756e+09,9.216313e+08,1.005692e+11,4.641123e+08
N2O,2.168365e+08,1.790871e+08,2.476122e+07,6.664511e+06,2.289607e+08,5.400237e+08,7.758891e+07,8.145946e+06,7.992618e+08,1.591878e+08,...,6.726210e+08,8.239212e+07,5.554300e+07,2.960401e+09,9.064638e+08,9.754589e+09,2.938656e+09,6.890047e+08,1.409893e+10,1.466206e+09


In [19]:
# Footprint calculation
e_CO2 = f_GHG.loc["CO2"] @ L @ Y_reg + F_hh_GHG_reg.loc["CO2"]
e_CH4 = f_GHG.loc["CH4"] @ L @ Y_reg + F_hh_GHG_reg.loc["CH4"]
e_N2O = f_GHG.loc["N2O"] @ L @ Y_reg + F_hh_GHG_reg.loc["N2O"]

e_CO2eq = e_CO2 + e_CH4 + e_N2O

# CO2_eq per capita
e_CO2eq_pp = e_CO2eq/pop2015.values/1000 #convert the unit from kg to metric ton/capita

e_CO2eq_pp

region
AT    12.756943
BE    15.889312
BG     7.072497
CY    11.636641
CZ    12.200267
DE    14.368607
DK    14.688278
EE    15.041655
ES     8.635758
FI    15.034909
FR     9.628982
GR    14.943025
HR     5.912843
HU     7.474584
IE    18.097559
IT    10.461834
LT     9.920193
LU    35.721233
LV     8.968882
MT    17.744231
NL    16.539988
PL     9.823864
PT     7.945518
RO     5.568849
SE    10.085219
SI    10.782638
SK    10.789558
GB    13.476931
US    22.644429
JP    11.901278
CN     8.036088
CA    18.070868
KR    15.049373
BR     5.494303
IN     2.603776
MX     5.076482
RU    10.337273
AU    26.364487
CH    14.816107
TR     7.262026
TW     9.246116
NO    18.087145
ID     3.342023
ZA     6.989603
WA     3.198766
WL     5.725800
WE     6.077016
WF     2.308441
WM     8.233952
dtype: float64

#### 1.3 What is the proportion of CO2 emissions in CO2e in each region's carbon footprint measured in CO2e? (4 points)

In [20]:
# Proportion of CO2 emissions to GWP by country
e_GWP = e_CO2 + e_CH4 + e_N2O
CO2_to_CO2eq = e_CO2/e_GWP * 100

CO2_to_CO2eq, CO2_to_CO2eq.max(), CO2_to_CO2eq.min() 

(region
 AT    76.911585
 BE    74.319444
 BG    71.715219
 CY    84.348826
 CZ    78.843490
 DE    80.717822
 DK    79.487342
 EE    75.628829
 ES    72.448500
 FI    68.435687
 FR    71.892165
 GR    83.724753
 HR    67.517701
 HU    72.929075
 IE    64.780323
 IT    73.292521
 LT    68.069377
 LU    64.853413
 LV    67.383587
 MT    72.849815
 NL    74.496797
 PL    75.320452
 PT    69.131867
 RO    68.760287
 SE    71.226852
 SI    69.798565
 SK    71.309414
 GB    72.490066
 US    81.778739
 JP    84.425858
 CN    80.082957
 CA    78.301802
 KR    81.198245
 BR    44.887583
 IN    62.569440
 MX    68.827817
 RU    77.043127
 AU    70.692367
 CH    75.767547
 TR    70.730723
 TW    74.318011
 NO    74.864554
 ID    55.152768
 ZA    79.644091
 WA    64.541373
 WL    62.614838
 WE    63.292054
 WF    37.225620
 WM    79.093796
 dtype: float64,
 84.42585759260712,
 37.22561994383321)

Double click on this cell to write your answer. Run the cell (ctrl+enter) once you are done writing so that it saves your input

## Question 2: Territorial emissions (6 points)

#### 2.1 What were the per capita territorial emissions of the US and China, respectively, in 2015? Only consider CO2 emissions here. (6 points)

In [21]:
e_CO2_pba = np.diag(f_sat_CO2) @ x
e_CO2_pba = e_CO2_pba.reshape(r,s).sum(1) + F_sat_hh_CO2_reg

e_CO2_pba_pp = e_CO2_pba/pop2015.values/1000 #convert unit from kg to tonne/capita

e_CO2_pba_pp.loc[["US","CN"]]

region
US    15.955813
CN     7.315289
dtype: float64

## Question 3: Emissions embodied in trade (20 points)

#### 3.1 Were the US and China net importers or exporters of CO2 emissions in 2015? (4 points)

In [22]:
net_import = (e_CO2 - e_CO2_pba)*1e-9 # convert from kg to million metric ton
net_import.loc[["US","CN"]]

region
US     826.367883
CN   -1194.159021
dtype: float64

Double click on this cell to write your answer. Run the cell (ctrl+enter) once you are done writing so that it saves your input

#### 3.2 Which were the top three regional contributers of US' and China's carbon footprints (only consider CO2 emissions), respectively, in 2015? (12 points)

In [23]:
e_CO2_cont = np.diag(f_sat_CO2) @ L @ Y_reg
e_CO2_US = e_CO2_cont.loc[:, "US"]
e_CO2_CN = e_CO2_cont.loc[:, "CN"]

e_CO2_US.index = Y.index
e_CO2_CN.index = Y.index

# print(e_CO2_US.groupby(level=0).sum().nlargest(n=3))
# print(e_CO2_CN.groupby(level=0).sum().nlargest(n=3))

print(e_CO2_US.groupby(level=0).sum().sort_values(ascending=False)[:3])
print(e_CO2_CN.groupby(level=0).sum().sort_values(ascending=False)[:3])

region
US    3.195043e+12
CN    3.970129e+11
CA    1.344926e+11
Name: US, dtype: float64
region
CN    7.495721e+12
WA    1.824185e+11
WM    9.450926e+10
Name: CN, dtype: float64


#### 3.3 By conducting emissions embodied in trade calculations, how much CO2 emissions did the US outsourced to China in 2015? (4 points)

In [24]:
e_CO2_US_to_CN = e_CO2_US.loc["CN"].sum() - e_CO2_CN.loc["US"].sum()
e_CO2_US_to_CN * 1e-9

338.84369064912204

The net outsourcing of CO2 emissions by the US to China in 2015 was 338.8 Mt CO2

## Question 4 Environmental vs Economic Footprints Analysis (16 points)

#### 4.1 Using the "impact" accounts, what were the value added footrpints of the US and China, respectively, in 2015? (8 points)

In [25]:
# Import impact accounts
F_imp = pd.read_csv(f'{path}impacts/F.txt' , sep='\t', index_col=[0], header=[0, 1])
F_imp_hh = pd.read_csv(f'{path}impacts/F_y.txt' , sep='\t', index_col=[0], header=[0, 1])

In [26]:
# VA coefficients
VA = F_imp[F_imp.index.str.contains("Value Added")]
f_VA = VA.values @ inv_diag_x_
e_VA = f_VA @ L @ Y_reg

e_VA_pp = e_VA/pop2015.values*1e6 

e_VA_pp.loc[:, ["US", "CN"]]
 

region,US,CN
0,52445.887193,7107.782845


#### 4.2 Make two main observations on the carbon vs value added footprint results regarding the relationships between the US and China? Please refer to the carbon footprint results obtained in Question 1 (8 points)

Double click on this cell to write your answer. Run the cell (ctrl+enter) once you are done writing so that it saves your input